In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import time
from datetime import datetime
import tensorflow_addons as tfa
import keras_tuner as kt

In [2]:
def exports():   
    # Set CUDA and CUPTI paths  
    os.environ['CUDA_HOME'] = '/usr/local/cuda'
    os.environ['PATH']= '/usr/local/cuda/bin:$PATH'  
    os.environ['CPATH'] = '/usr/local/cuda/include:$CPATH'  
    os.environ['LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/extras/CUPTI/lib64:$LD_LIBRARY_PATH'  
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LD_LIBRARY_PATH'
    os.environ['TF_GPU_THREAD_MODE']='gpu_private'

In [3]:
exports

<function __main__.exports()>

# Load data

In [4]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gf34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf34.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
hc10= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/hc10.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
no22= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/no22.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo27= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo27.csv', low_memory=False, header=None)
wo34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo34.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)
wo37= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo37.csv', low_memory=False, header=None)
wo40= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo40.csv', low_memory=False, header=None)
wo45= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo45.csv', low_memory=False, header=None)

In [5]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01_EarlyMarch.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36_EarlyMarch.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)

In [6]:
frames =[cf15, gf25,gm01,nc46, nc74,nf11, wc16, wc41,wo27,wo34, wo36, wo37, wo40, wo45]
frames_2=[val_1, val_2, val_3]
data = pd.concat(frames)
val = pd.concat(frames_2)

# Define data cleaning functions

In [7]:
def snip(data):
    #Snip unwanted characters 

    itime= data.iloc[:,0]
    itime=itime.map(lambda x: str(x[6:]))
    itime=itime.map(lambda x: float(x))

    date=itime
    minute=date.map(lambda x: time.gmtime(x+7200)[4])
    hour=date.map(lambda x: time.gmtime(x+7200)[3])
    day=date.map(lambda x: time.gmtime(x+7200)[2])
    month=date.map(lambda x: time.gmtime(x+7200)[1])

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    dev_id=data.iloc[:,3]
    dev_id=dev_id.map(lambda x: str(x))    
    dev_id=dev_id.map(lambda x: str(x[7:]))
    dev_id=dev_id.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    status=status.rolling(10).mean()
    status=status.map(np.floor)

    train_init=pd.concat([itime, minute, hour, day, month, interface, dev_id, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time','Minute', 'Hour','Day','Month', 'Interface', 'Dev_id', 'Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    #Remove unwanted columns
    train_label=train.filter(['Status'])
    train=train.drop(['Status','Inband', 'Outband'], axis=1)
    train= train.sort_values(by=['Dev_id', 'Time'])
    train=train.reset_index(drop=True)
    train=train.drop(['Dev_id'], axis=1)

    return train, train_label

In [8]:
train,train_label=snip(data)
train=train.iloc[:-700, :]
train=train.to_numpy()
train_label=train_label.iloc[700:]

In [9]:
test,test_label=snip(val)
test=test.iloc[:-700, :]
test=test.to_numpy()
test_label=test_label.iloc[700:]

In [10]:
#train, train_label = preproccess(data)
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std
#test, test_label = preproccess(val)
test-=mean
test/=std

In [11]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

2022-03-13 19:00:41.843731: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-13 19:00:41.843770: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-13 19:00:41.908093: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-03-13 19:00:41.908917: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-03-13 19:00:41.958750: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-13 19:00:41.958866: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


In [12]:
def get_model(hp):
    model=tf.keras.Sequential()
    #Tune number of layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            tf.keras.layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=240, step=16),
                activation='tanh' #hp.Choice('activation',['elu','relu','tanh'])
            )
        )
        if hp.Boolean("dropout"):
            model.add(tf.keras.layers.Dropout(rate=hp.Choice('rate',[0.3,0.4,0.45,0.5])))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='BinaryCrossentropy',
                metrics=tfa.metrics.F1Score(num_classes=1, average='micro'))
    
    return model


In [13]:
tuner = kt.Hyperband(get_model,
                     objective=kt.Objective("val_f1_score", direction="max"),
                     max_epochs=7,
                     factor=3,
                     hyperband_iterations=1,
                     directory='/home/jordan/Documents/DataSets/Data-Science/test7',
                     project_name='Tuning_test')

2022-03-13 19:00:42.139505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:00:42.239556: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:00:42.239839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-13 19:00:42.241114: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [14]:
callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
tuner.search(train, train_label, epochs=30, validation_split=0.2,
callbacks= [callback])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 03m 13s]
val_f1_score: 0.011293267831206322

Best val_f1_score So Far: 0.8461083769798279
Total elapsed time: 00h 23m 52s
INFO:tensorflow:Oracle triggered exit


In [16]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train, train_label, epochs=20, validation_split=0.2)

Epoch 1/20
17259/17259 [==============================] - 39s 2ms/step - loss: 7.1697 - f1_score: 0.1148 - val_loss: 3.3850 - val_f1_score: 0.2323
Epoch 2/20
17259/17259 [==============================] - 39s 2ms/step - loss: 6.8946 - f1_score: 0.2907 - val_loss: 4.3807 - val_f1_score: 0.7251
Epoch 3/20
17259/17259 [==============================] - 38s 2ms/step - loss: 6.7974 - f1_score: 0.4989 - val_loss: 4.0872 - val_f1_score: 0.7604
Epoch 4/20
17259/17259 [==============================] - 41s 2ms/step - loss: 6.7379 - f1_score: 0.5457 - val_loss: 3.8512 - val_f1_score: 0.8062
Epoch 5/20
17259/17259 [==============================] - 41s 2ms/step - loss: 6.7047 - f1_score: 0.5671 - val_loss: 4.0269 - val_f1_score: 0.7996
Epoch 6/20
17259/17259 [==============================] - 37s 2ms/step - loss: 6.6888 - f1_score: 0.5742 - val_loss: 4.1716 - val_f1_score: 0.7978
Epoch 7/20
17259/17259 [==============================] - 40s 2ms/step - loss: 6.6707 - f1_score: 0.5839 - val_loss: 4

In [17]:
val_acc_per_epoch = history.history["val_f1_score"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 11


In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

#Retrain the model
hypermodel.fit(train, train_label, epochs=best_epoch, validation_split=0.2)

Epoch 1/11
17259/17259 [==============================] - 42s 2ms/step - loss: 7.0934 - f1_score: 0.1165 - val_loss: 3.9835 - val_f1_score: 0.2790
Epoch 2/11
17259/17259 [==============================] - 37s 2ms/step - loss: 6.8377 - f1_score: 0.3399 - val_loss: 3.5812 - val_f1_score: 0.7535
Epoch 3/11
17259/17259 [==============================] - 36s 2ms/step - loss: 6.7944 - f1_score: 0.5431 - val_loss: 3.4740 - val_f1_score: 0.8373
Epoch 4/11
17259/17259 [==============================] - 42s 2ms/step - loss: 6.7610 - f1_score: 0.5860 - val_loss: 3.8909 - val_f1_score: 0.8469
Epoch 5/11
17259/17259 [==============================] - 38s 2ms/step - loss: 6.7605 - f1_score: 0.5941 - val_loss: 3.9488 - val_f1_score: 0.8458
Epoch 6/11
17259/17259 [==============================] - 39s 2ms/step - loss: 6.7409 - f1_score: 0.6012 - val_loss: 4.1645 - val_f1_score: 0.8313
Epoch 7/11
17259/17259 [==============================] - 36s 2ms/step - loss: 6.7484 - f1_score: 0.6008 - val_loss: 4

In [19]:
eval_result = hypermodel.evaluate(test, test_label)
print("[test loss, test accuracy]:", eval_result)

2910/2910 [==============================] - 4s 2ms/step - loss: 0.7965 - f1_score: 0.9732
[test loss, test accuracy]: [0.796463668346405, 0.9731848835945129]


In [20]:
hypermodel.save('tuned_model7.h5')

In [21]:
hypermodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                800       
                                                                 
 dropout_3 (Dropout)         (None, 80)                0         
                                                                 
 dense_5 (Dense)             (None, 224)               18144     
                                                                 
 dropout_4 (Dropout)         (None, 224)               0         
                                                                 
 dense_6 (Dense)             (None, 16)                3600      
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
Total params: 22,544
Trainable params: 22,544
Non-trai